In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
engine = create_engine(
    "mysql+pymysql://root:@localhost:3306/accounting_system_myself"
)


In [3]:
users = pd.read_sql("""
    SELECT id, name, full_name, phone, email, branch_id,image,password, created_at
    FROM users
""", engine)


In [23]:
branches = pd.read_sql("""
    SELECT id, activity, phone AS branch_phone, email AS branch_email,image As branch_image,fax,deleted_at
    FROM branches
""", engine)


In [24]:
branch_details = pd.read_sql("""
    SELECT branch_id, locale, name AS branch_name, address
    FROM branch_translations
""", engine)


In [6]:
users_branches = pd.merge(
    users,
    branches,
    left_on="branch_id",
    right_on="id",
    how="left",
    suffixes=("_user", "_branch")
)


In [7]:
final_df = pd.merge(
    users_branches,
    branch_details,
    on="branch_id",
    how="left"
)


In [8]:
users_ar = final_df[final_df["locale"] == "ar"]


In [17]:
users_ar[['full_name','name','phone','email','branch_name','image']]


,full_name,name,phone,email,branch_name,image
0,mariam omar,mariam,774548558,admin@admin.com,نظام محاسبي,User/accounting_system_1768142915.png
2,Mariam Babaqi,Mariam Babaqi,None,mariambabaqi@gmail.com,نظام محاسبي,User/accounting_system1768125712.png
4,test,test,147852,admin@test.com,نظام محاسبي,None
6,Laravel,Laravel,None,phplaravel1990@gmail.com,نظام محاسبي,None


In [25]:
branch = pd.merge(
    branches,
    branch_details,
    left_on="id",
    right_on="branch_id",
    how="left",

)

In [26]:
branch_ar = branch[branch["locale"] == "ar"]


In [28]:
branch_not_deleted = branch_ar[branch_ar["deleted_at"].isna()]


In [30]:
branch_not_deleted[['branch_name','address','activity','branch_email','branch_phone','fax','branch_image']]

,branch_name,address,activity,branch_email,branch_phone,fax,branch_image
0,نظام محاسبي,صنعاء,None,admin@admin.com,774548558,4548558,branch/accounting_system_1761216291.png
2,الفرع الرئيسي,صنعاء - شارع حدة,شركة محاسبة و تجارة,branch1@example.com,777100001,1,branch1.jpg
4,الفرع الفرعي,تعز - شارع الستين,شركة محاسبة و تجارة,branch2@example.com,777100002,1,branch2.jpg


In [31]:
funds = pd.read_sql("""
    SELECT id, account_id, deleted_at
    FROM funds
""", engine)

In [32]:
fund_translations = pd.read_sql("""
    SELECT id AS fund_translations_id , name, locale,fund_id
    FROM fund_translations
""", engine)

In [33]:
fund = pd.merge(
    funds,
    fund_translations,
    left_on="id",
    right_on="fund_id",
    how="left",
    suffixes=("_fund", "_fund_translations")
)

In [34]:
fund.head()

,id,account_id,deleted_at,fund_translations_id,name,locale,fund_id
0,1,6,None,1,الصندوق الرئيسي,ar,1
1,1,6,None,2,Main Cash Fund,en,1
2,2,6,None,3,صندوق الطوارئ,ar,2
3,2,6,None,4,Emergency Cash Fund,en,2


In [35]:
accounts = pd.read_sql("""
    SELECT *
    FROM accounts
""", engine)

In [36]:
accounts.head()

,id,ranks,account_id,acc_code,acc_type_id,acc_report_type_id,acc_level,acc_debit,acc_credit,acc_balance,userAdd,userEdit,branch_id,status,created_at,updated_at,deleted_at
0,1,1,NaN,1000,1,1,1,0,0,0,1.0,None,1,1,2026-01-13 11:24:00,NaT,None
1,2,2,NaN,2000,1,1,1,0,0,0,1.0,None,1,1,2026-01-13 11:24:00,NaT,None
2,3,3,NaN,3000,1,1,1,0,0,0,1.0,None,1,1,2026-01-13 11:24:00,NaT,None
3,4,4,NaN,4000,1,2,1,0,0,0,1.0,None,1,1,2026-01-13 11:24:00,NaT,None
4,5,5,NaN,5000,1,2,1,0,0,0,1.0,None,1,1,2026-01-13 11:24:00,NaT,None


In [40]:
account_translations = pd.read_sql("""
    SELECT id AS account_translations , name AS account_name, locale,account_id
    FROM account_translations
""", engine)

In [41]:
fund_account = pd.merge(
    fund,
    accounts,
    left_on="account_id",
    right_on="id",
    how="left",
    suffixes=("_fund", "_accounts")
)

In [42]:
fund_account.head()

,id_fund,account_id_fund,deleted_at_fund,fund_translations_id,name,locale,fund_id,id_accounts,ranks,account_id_accounts,...,acc_debit,acc_credit,acc_balance,userAdd,userEdit,branch_id,status,created_at,updated_at,deleted_at_accounts
0,1,6,None,1,الصندوق الرئيسي,ar,1,6,1,1.0,...,0,0,0,1.0,None,1,1,2026-01-13 11:24:08,NaT,None
1,1,6,None,2,Main Cash Fund,en,1,6,1,1.0,...,0,0,0,1.0,None,1,1,2026-01-13 11:24:08,NaT,None
2,2,6,None,3,صندوق الطوارئ,ar,2,6,1,1.0,...,0,0,0,1.0,None,1,1,2026-01-13 11:24:08,NaT,None
3,2,6,None,4,Emergency Cash Fund,en,2,6,1,1.0,...,0,0,0,1.0,None,1,1,2026-01-13 11:24:08,NaT,None


In [43]:
fund_account_all = pd.merge(
    fund_account,
    account_translations,
    left_on="account_id_fund",
    right_on="account_id",
    how="left",
    suffixes=("_fund", "_accounts")
)

In [44]:
fund_account_all.head()

,id_fund,account_id_fund,deleted_at_fund,fund_translations_id,name,locale_fund,fund_id,id_accounts,ranks,account_id_accounts,...,userEdit,branch_id,status,created_at,updated_at,deleted_at_accounts,account_translations,account_name,locale_accounts,account_id
0,1,6,None,1,الصندوق الرئيسي,ar,1,6,1,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,11,الصندوق,ar,6
1,1,6,None,1,الصندوق الرئيسي,ar,1,6,1,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,12,Cash,en,6
2,1,6,None,2,Main Cash Fund,en,1,6,1,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,11,الصندوق,ar,6
3,1,6,None,2,Main Cash Fund,en,1,6,1,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,12,Cash,en,6
4,2,6,None,3,صندوق الطوارئ,ar,2,6,1,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,11,الصندوق,ar,6


In [46]:
final_fund_account_all = fund_account_all[fund_account_all["deleted_at_fund"].isna()]


In [70]:
fund_ar = final_fund_account_all[final_fund_account_all["locale_fund"] == "ar"]


In [71]:
account_fund_ar = fund_ar[fund_ar["locale_accounts"] == "ar"]

In [72]:
account_fund_ar[['name','account_name','acc_code']]

,name,account_name,acc_code
0,الصندوق الرئيسي,الصندوق,1100
4,صندوق الطوارئ,الصندوق,1100


In [50]:
banks = pd.read_sql("""
    SELECT id, account_id, deleted_at
    FROM banks
""", engine)

In [52]:
bank_translations = pd.read_sql("""
    SELECT id AS bank_translations_id , name, locale,bank_id
    FROM bank_translations
""", engine)

In [53]:
bank = pd.merge(
    banks,
    bank_translations,
    left_on="id",
    right_on="bank_id",
    how="left",
    suffixes=("_bank", "_bank_translations")
)

In [54]:
bank.head()

,id,account_id,deleted_at,bank_translations_id,name,locale,bank_id
0,1,7,None,1,البنك الرئيسي,ar,1
1,1,7,None,2,Main Bank,en,1
2,2,7,None,3,بنك التوفير,ar,2
3,2,7,None,4,Savings Bank,en,2


In [55]:
bank_account = pd.merge(
    bank,
    accounts,
    left_on="account_id",
    right_on="id",
    how="left",
    suffixes=("_bank", "_accounts")
)

In [56]:
bank_account_all = pd.merge(
    bank_account,
    account_translations,
    left_on="account_id_bank",
    right_on="account_id",
    how="left",
    suffixes=("_bank", "_accounts")
)

In [57]:
final_bank_account_all = bank_account_all[bank_account_all["deleted_at_bank"].isna()]


In [63]:
bank_ar = final_bank_account_all[final_bank_account_all["locale_bank"] == "ar"]


In [67]:
bank_ar.head()

,id_bank,account_id_bank,deleted_at_bank,bank_translations_id,name,locale_bank,bank_id,id_accounts,ranks,account_id_accounts,...,userEdit,branch_id,status,created_at,updated_at,deleted_at_accounts,account_translations,account_name,locale_accounts,account_id
0,1,7,None,1,البنك الرئيسي,ar,1,7,2,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,13,البنك,ar,7
1,1,7,None,1,البنك الرئيسي,ar,1,7,2,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,14,Bank,en,7
4,2,7,None,3,بنك التوفير,ar,2,7,2,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,13,البنك,ar,7
5,2,7,None,3,بنك التوفير,ar,2,7,2,1.0,...,None,1,1,2026-01-13 11:24:08,NaT,None,14,Bank,en,7


In [68]:
account_bank_ar = bank_ar[bank_ar["locale_accounts"] == "ar"]

In [73]:
account_bank_ar[['name','account_name','acc_code']]

,name,account_name,acc_code
0,البنك الرئيسي,البنك,1200
4,بنك التوفير,البنك,1200


In [74]:
currencies = pd.read_sql("""
    SELECT *
    FROM currencies
""", engine)

In [75]:
currency_translations = pd.read_sql("""
    SELECT *
    FROM currency_translations
""", engine)

In [76]:
currency = pd.merge(
    currencies,
    currency_translations,
    left_on="id",
    right_on="currency_id",
    how="left",
    suffixes=("_currency", "_currency_translations")
)

In [77]:
currency.head()

,id_currency,symbol,currency_type_id,curr_val,curr_penny,created_at,updated_at,deleted_at,id_currency_translations,currency_id,locale,name
0,1,YER,1,1,100,2026-01-13 11:38:37,None,None,1,1,ar,ريال يمني
1,1,YER,1,1,100,2026-01-13 11:38:37,None,None,2,1,en,Yemeni Rial


In [78]:
currency_types = pd.read_sql("""
    SELECT *
    FROM currency_types
""", engine)

In [79]:
currency_type_translations = pd.read_sql("""
    SELECT *
    FROM currency_type_translations
""", engine)

In [80]:
currency_type = pd.merge(
    currency_types,
    currency_type_translations,
    left_on="id",
    right_on="currency_type_id",
    how="left",
    suffixes=("_currency_type", "_currency_type_translations")
)

In [83]:
currency_type.head()

,id_currency_type,created_at,updated_at,deleted_at,id_currency_type_translations,currency_type_id,locale,name
0,1,2025-10-15 11:17:13,2025-10-15 11:17:13,None,1,1,ar,محلية
1,1,2025-10-15 11:17:13,2025-10-15 11:17:13,None,2,1,en,Local
2,2,2025-10-15 11:17:13,2025-10-15 11:17:13,None,3,2,ar,أجنبية
3,2,2025-10-15 11:17:13,2025-10-15 11:17:13,None,4,2,en,Foreign


In [84]:
currencies_type = pd.merge(
    currency,
    currency_type,
    left_on="currency_type_id",
    right_on="currency_type_id",
    how="left",
    suffixes=("_currencies", "_currency_types")
)

In [85]:
currencies_type.head()

,id_currency,symbol,currency_type_id,curr_val,curr_penny,created_at_currencies,updated_at_currencies,deleted_at_currencies,id_currency_translations,currency_id,locale_currencies,name_currencies,id_currency_type,created_at_currency_types,updated_at_currency_types,deleted_at_currency_types,id_currency_type_translations,locale_currency_types,name_currency_types
0,1,YER,1,1,100,2026-01-13 11:38:37,None,None,1,1,ar,ريال يمني,1,2025-10-15 11:17:13,2025-10-15 11:17:13,None,1,ar,محلية
1,1,YER,1,1,100,2026-01-13 11:38:37,None,None,1,1,ar,ريال يمني,1,2025-10-15 11:17:13,2025-10-15 11:17:13,None,2,en,Local
2,1,YER,1,1,100,2026-01-13 11:38:37,None,None,2,1,en,Yemeni Rial,1,2025-10-15 11:17:13,2025-10-15 11:17:13,None,1,ar,محلية
3,1,YER,1,1,100,2026-01-13 11:38:37,None,None,2,1,en,Yemeni Rial,1,2025-10-15 11:17:13,2025-10-15 11:17:13,None,2,en,Local


In [92]:
currency_not_null = currencies_type[currencies_type["deleted_at_currencies"].isna()]


In [98]:
currency_ar = currency_not_null[
    (currency_not_null["locale_currencies"].fillna("") == "ar") &
    (currency_not_null["locale_currency_types"].fillna("") == "ar")
]


In [99]:
currency_ar[['id_currency','name_currencies','name_currency_types','curr_penny','symbol','curr_val']]

,id_currency,name_currencies,name_currency_types,curr_penny,symbol,curr_val
0,1,ريال يمني,محلية,100,YER,1
